In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../vacation_search/weatherpy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Sakakah,SA,67.57,broken clouds,29.9697,40.2064,Fakhamat Al Orjoana Apartment
1,1,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
2,2,Hithadhoo,MV,82.90,overcast clouds,-0.6000,73.0833,Scoop Guest House
3,3,Salinopolis,BR,78.10,broken clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
4,5,Cape Town,ZA,62.89,scattered clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City'] == 'Mayo']
vacation_end = vacation_df.loc[vacation_df['City'] == 'Mayo']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Hurricane']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'San Patricio'] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Cheney'] 
vacation_stop3

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
64,117,Cheney,US,63.99,overcast clouds,47.4874,-117.5758,"Holiday Inn Express & Suites Cheney, an IHG Hotel"


In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.iloc[0,5:7].to_numpy()
end = vacation_end.iloc[0,5:7].to_numpy()
stop1 = vacation_stop1.iloc[0,5:7].to_numpy()
stop2 = vacation_stop2.iloc[0,5:7].to_numpy()
stop3 = vacation_stop3.iloc[0,5:7].to_numpy()

start

array([38.8876, -76.5119], dtype=object)

In [7]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure(center=(38, -122), zoom_level = 7)
Mayo = start
Hurricane = stop1
San_Patricio = stop2
Cheney = stop3 
roadtrip = gmaps.directions_layer(Mayo, Mayo, 
                                  waypoints=[Hurricane, San_Patricio, Cheney],
                                  travel_mode="DRIVING")

In [8]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, 
                          vacation_stop2, vacation_stop3]
                         ,ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,564,Mayo,US,63.55,overcast clouds,38.8876,-76.5119,Resorts Resource Group International
1,30,Hurricane,US,75.54,clear sky,37.1753,-113.2899,Super 8 by Wyndham Hurricane Zion National Park
2,83,San Patricio,US,84.29,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
3,117,Cheney,US,63.99,overcast clouds,47.4874,-117.5758,"Holiday Inn Express & Suites Cheney, an IHG Hotel"


In [9]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [10]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(marker_layer)
fig.add_layer(roadtrip)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))